### Introduction to Machine Learning: Clustering
#### Corey Solitaire
#### 10/09/2020

In [1]:
import acquire
import prepare
# import src.explore
# import src.model
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression, SelectKBest, RFE 
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

### Acquire & Summarize

#### 1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

    - Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
    - Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
    - Only include properties that include a latitude and longitude value.

In [2]:
# Acquire df
df = acquire.get_zillow_data(cached=True)
#pd.set_option('display.max_rows', df.shape[0]+1) # Shows all the rows

In [3]:
df.shape

(77381, 70)

In [4]:
# Rename duplicate column id/id with id_delete/id
df.columns = ['parcelid', 'typeconstructiontypeid', 'storytypeid',
       'propertylandusetypeid', 'heatingorsystemtypeid', 'buildingclasstypeid',
       'architecturalstyletypeid', 'airconditioningtypeid', 'id_delete',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
       'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertyzoningdesc',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
       'numberofstories', 'fireplaceflag', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'taxdelinquencyflag', 'taxdelinquencyyear',
       'censustractandblock', 'id', 'logerror', 'pid', 'tdate',
       'airconditioningdesc', 'architecturalstyledesc', 'buildingclassdesc',
       'heatingorsystemdesc', 'propertylandusedesc', 'storydesc',
       'typeconstructiondesc']

In [5]:
# Delete duplicate columns id_delete and pid (dupilcate of parcelid)
df.drop(columns = ['id_delete','pid'], inplace = True)
df.shape

(77381, 68)

***

#### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

***

In [6]:
def df_summary(df):
    print('The shape of the df:') 
    print(df.shape)  # df shape (row/column)
    print('\n')
    print('Columns, Non-Null Count, Data Type:')
    print(df.info())      # Column, Non Null Count, Data Type
    print('\n')
    print('Summary statistics for the df:') 
    print(df.describe())             # Summary Statistics on Numeric Data Types
    print('\n')
    print('Number of NaN values per column:') 
    print(df.isna().sum())           # NaN by column
    print('\n')
    print('Number of NaN values per row:')  
    print(df.isnull().sum(axis=1))   # NaN by row
    print('\n')
    print('Value Counts per Column:')
    for col in df.columns:
        print('-' * 40 + col + '-' * 40 , end=' - ')
        display(df[col].value_counts(dropna=False).head(10))
        #display(df_resp[col].value_counts())  # Displays all Values, not just First 10

In [7]:
df_summary(df)

The shape of the df:
(77381, 68)


Columns, Non-Null Count, Data Type:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77381 non-null  int64  
 1   typeconstructiontypeid        222 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77381 non-null  float64
 4   heatingorsystemtypeid         49440 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      206 non-null    float64
 7   airconditioningtypeid         24953 non-null  float64
 8   basementsqft                  50 non-null     float64
 9   bathroomcnt                   77381 non-null  float64
 10  bedroomcnt                    77381 non-null  float64
 11  buildingqualitytypeid         49672 non-null  fl

12086286    1
13842812    1
17089101    1
14604878    1
11149905    1
12104347    1
11077074    1
17173076    1
12714581    1
12458582    1
Name: parcelid, dtype: int64

----------------------------------------typeconstructiontypeid---------------------------------------- - 

NaN     77159
6.0       219
4.0         1
13.0        1
10.0        1
Name: typeconstructiontypeid, dtype: int64

----------------------------------------storytypeid---------------------------------------- - 

...

Single Family Residential                     52320
Condominium                                   19294
Duplex (2 Units, Any Combination)              2009
Planned Unit Development                       1944
Quadruplex (4 Units, Any Combination)           727
Triplex (3 Units, Any Combination)              535
Cluster Home                                    333
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       58
Residential General                              37
Name: propertylandusedesc, dtype: int64

----------------------------------------storydesc---------------------------------------- - 

NaN         77331
Basement       50
Name: storydesc, dtype: int64

----------------------------------------typeconstructiondesc---------------------------------------- - 

NaN         77159
Frame         219
Concrete        1
Metal           1
Masonry         1
Name: typeconstructiondesc, dtype: int64

***

#### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.*

***

In [8]:
def count_and_percent_missing_row(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    total_missing = df.isnull().sum()
    missing_value_df = pd.DataFrame({'num_rows_missing': total_missing,
                                     'pct_rows_missing': percent_missing})
    return missing_value_df

In [9]:
count_and_percent_missing_row(df)

,num_rows_missing,pct_rows_missing
parcelid,0,0.000000
typeconstructiontypeid,77159,99.713108
storytypeid,77331,99.935385
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,27941,36.108347
...,...,...
buildingclassdesc,77366,99.980615
heatingorsystemdesc,27941,36.108347
propertylandusedesc,0,0.000000
storydesc,77331,99.935385


***

#### Based on this I would want to delete attributes that have less than 50% of their data missing.

***

***

#### 4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

***

In [10]:
def count_and_percent_missing_column(df):
    num_rows = df.loc[:].isnull().sum()
    num_cols_missing = df.loc[:, df.isna().any()].count()
    pct_cols_missing = round(df.loc[:, df.isna().any()].count() / len(df.index) * 100, 3)
    missing_cols_and_rows_df = pd.DataFrame({'num_cols_missing': num_cols_missing,
                                             'pct_cols_missing': pct_cols_missing,
                                             'num_rows': num_rows})
    missing_cols_and_rows_df = missing_cols_and_rows_df.fillna(0)
    missing_cols_and_rows_df['num_cols_missing'] = missing_cols_and_rows_df['num_cols_missing'].astype(int)
    return missing_cols_and_rows_df

In [11]:
count_and_percent_missing_column(df)

,num_cols_missing,pct_cols_missing,num_rows
airconditioningdesc,24953,32.247,52428
airconditioningtypeid,24953,32.247,52428
architecturalstyledesc,206,0.266,77175
architecturalstyletypeid,206,0.266,77175
assessmentyear,0,0.000,0
...,...,...,...
typeconstructiontypeid,222,0.287,77159
unitcnt,50563,65.343,26818
yardbuildingsqft17,2388,3.086,74993
yardbuildingsqft26,70,0.090,77311


***

#### Based on this I would want to delete columns that have less than 75% of their data missing.

***

### Prepare

#### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.


Property Codes that Fit the Breif of (Single Unit Properties)

propertylandusetypeid
-        261          Single Family Residential
-        263          Mobile Home
-        264          Townhouse
-        266          Condominium
-        273          Bungalow
-        276          Patio Home
-        275          Manufactured, Modular, Prefabricated Homes
-        279          Inferred Single Family Residential

Alternate Methods to determine single Unit Properties

df.unitcnt == 1
df.roomcnt > 0
df.garagecarcnt > 0
df.bedroomcnt > 0
df.bathroomcnt > 0

In [12]:
df.shape

(77381, 68)

In [49]:
df = acquire.get_zillow_data(cached=True)
unit_df = df[df['unitcnt']==1]
unit_df.shape
# df = df[df['roomcnt']>0]
# df = df[df['garagecarcnt']>0]
# df = df[df['bedroomcnt']>0]
# df = df[df['bathroomcnt']>0]

(47293, 70)

In [50]:
df = acquire.get_zillow_data(cached=True)
room_df = df[df['roomcnt']>0]
room_df.shape

(17573, 70)

In [51]:
df = acquire.get_zillow_data(cached=True)
garage_df = df[df['garagecarcnt']>0]
garage_df.shape

(25244, 70)

In [52]:
df = acquire.get_zillow_data(cached=True)
bed_df = df[df['bedroomcnt']>0]
bed_df.shape

(76551, 70)

In [53]:
df = acquire.get_zillow_data(cached=True)
bath_df = df[df['bathroomcnt']>0]
bath_df.shape

(76789, 70)

In [60]:
# Unique dataframe that does not use proptertylandusetypeid to esimate single unit properties
new_df = pd.concat([bed_df, bath_df, garage_df, room_df, unit_df]).drop_duplicates('id').reset_index(drop=True)
new_df.shape

(76871, 70)

In [63]:
#pd.set_option('display.max_rows', df.shape[0]+1) # Shows all the rows
new_df.parcelid.value_counts(ascending=False)

12086286     1
11079752     1
14481994     1
13842812     1
17229443     1
17089101     1
14604878     1
11149905     1
12104347     1
11077074     1
17173076     1
12714581     1
12458582     1
12063319     1
11561560     1
10742616     1
11956827     1
12640841     1
11174469     1
11819614     1
14201411     1
17055835     1
13095471     1
14719290     1
14738127     1
14627379     1
17096515     1
14383674     1
17265211     1
10832444     1
12927549     1
11754046     1
13849151     1
12708971     1
10920513     1
11844162     1
11400644     1
12702184     1
14631805     1
14650724     1
11971200     1
11430996     1
14596738     1
12558694     1
12346649     1
12949950     1
17097353     1
14130263     1
11979404     1
12894861     1
13947535     1
10982486     1
17054354     1
14444701     1
11012756     1
12538495     1
11750012     1
12960605     1
12765506     1
11002372     1
12948071     1
11266664     1
11005825     1
13908557     1
10867311     1
12266096     1
11189106  

### Prep Data Function:

In [15]:
# Function to Prep Data
def data_prep(df, cols_to_remove=[], prop_required_column=.5, prop_required_row=.75):
    
    def remove_columns(df, cols_to_remove):  
        df = df.drop(columns=cols_to_remove)
        return df

    def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
        threshold = int(round(prop_required_column*len(df.index),0))
        df.dropna(axis=1, thresh=threshold, inplace=True)
        threshold = int(round(prop_required_row*len(df.columns),0))
        df.dropna(axis=0, thresh=threshold, inplace=True)
        return df
    
    df = remove_columns(df, cols_to_remove)  # Removes Specified Columns
    df = handle_missing_values(df, prop_required_column, prop_required_row) # Removes Specified Rows
    #df.dropna(inplace=True) # Drops all Null Values From Dataframe
    return df


# How to Call the Function
# df = prepare.data_prep(
#     df,
#     cols_to_remove=[],
#     prop_required_column=.6,
#     prop_required_row=.75
# )